# Übung 05 - Immobilienpreise 
### <span style="color:CornflowerBlue">Abgabetermin: 19.06.2023</span>
### <span style="color:CornflowerBlue">Speichern Sie die Übung im Ordner 06-Uebung in Ihrem Git.</span>

In dieser Übung sollen Sie Immobilienpreise mit Hilfe von numerischen und kategorischen Merkmalen aus einem unvollständigen Datensatz mit einem Random-Forest-Regressor schätzen. Zusätzlich sollen mit Hilfe von Kreuzvalidierung Anzahl an Entscheidungsbäumen bestimmen, die das beste Ergebnis liefert.


Original Daten: [Iowa, Ames Housing Datensatz](https://ww2.amstat.org/publications/jse/v19n3/decock/AmesHousing.txt)

De Cock, D., 2011. Ames, Iowa: Alternative to the Boston housing data as an end of semester regression project. Journal of Statistics Education, 19(3).

In [85]:
# importieren der Bibliotheken
import numpy as np
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.metrics import mean_absolute_error, r2_score

# plots direkt im notebook anzeigen
%matplotlib inline

a) Lesen Sie den Datensatz ein. (Laden Sie die Daten mit index_col="Id")

In [86]:
# your code here
df = pd.read_csv("data.csv", index_col="Id")
df.head(4)


,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000


b) Prüfen Sie die Daten auf fehlende Werte.

In [87]:
# your code here
df.isnull().values.any()
display(df.head())

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000


c) Entfernen Sie die Merkmale (Spalten) mit zu vielen fehlenden Werten. (Zum Beispiel: > 500, > 1000)

In [88]:
# your code here

nan_counts=df.isna().sum()
columns_to_drop = nan_counts[nan_counts > 500].index
df = df.drop(columns=columns_to_drop)
display(df.head())




,MSSubClass,MSZoning,LotFrontage,LotArea,Street,LotShape,LandContour,Utilities,LotConfig,LandSlope,...,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
Id,,,,,,,,,,,,,,,,,,,,,
1,60,RL,65.0,8450,Pave,Reg,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,2,2008,WD,Normal,208500
2,20,RL,80.0,9600,Pave,Reg,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,5,2007,WD,Normal,181500
3,60,RL,68.0,11250,Pave,IR1,Lvl,AllPub,Inside,Gtl,...,0,0,0,0,0,9,2008,WD,Normal,223500
4,70,RL,60.0,9550,Pave,IR1,Lvl,AllPub,Corner,Gtl,...,272,0,0,0,0,2,2006,WD,Abnorml,140000
5,60,RL,84.0,14260,Pave,IR1,Lvl,AllPub,FR2,Gtl,...,0,0,0,0,0,12,2008,WD,Normal,250000


d) Aufteilen der Daten in Merkmale (X) und Zielvariable (y)

In [89]:
# your code here
X = df.loc[ : , df.columns != 'SalePrice']
#X = X.loc[ : , df.columns != 'Id']
y= df['SalePrice']

e) Unterteilen Sie den Datensatz in Trainings- und Testdaten. Teilen Sie die Daten im Verhältnis 80% Trainingsdaten und 30% Testdaten auf.

In [90]:
# your code here
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(X_train)

      MSSubClass MSZoning  LotFrontage  LotArea Street LotShape LandContour  \
Id                                                                            
255           20       RL         70.0     8400   Pave      Reg         Lvl   
1067          60       RL         59.0     7837   Pave      IR1         Lvl   
639           30       RL         67.0     8777   Pave      Reg         Lvl   
800           50       RL         60.0     7200   Pave      Reg         Lvl   
381           50       RL         50.0     5000   Pave      Reg         Lvl   
...          ...      ...          ...      ...    ...      ...         ...   
1096          20       RL         78.0     9317   Pave      IR1         Lvl   
1131          50       RL         65.0     7804   Pave      Reg         Lvl   
1295          20       RL         60.0     8172   Pave      Reg         Lvl   
861           50       RL         55.0     7642   Pave      Reg         Lvl   
1127         120       RL         53.0     3684   Pa

f) Befüllen Sie die fehlenden Werte der numerischen Merkmale. Wählen sie hierzu eine geeignete Strategie.

Tip: SimpleImputer(strategy=["mean"|"median"|"most_frequent"|"constant"]) aus sklearn.

In [91]:
# your code here
imputer=SimpleImputer(missing_values=np.nan, strategy="mean")
numeric_columns = X_train.select_dtypes(include=['int', 'float']).columns

# Fehlende Werte in den Trainingsdaten ersetzen
X_train[numeric_columns] = imputer.fit_transform(X_train[numeric_columns])

# Fehlende Werte in den Testdaten ersetzen
X_test[numeric_columns] = imputer.transform(X_test[numeric_columns])

g) Befüllen Sie die fehlenden Werte der kategorischen Variablen. Verwenden Sie hierfür den Wert, der am häufigsten in der Spalte vorkommt.

In [92]:
# your code here
imputer2=SimpleImputer(missing_values=np.nan, strategy="most_frequent")
categoric_columns = X.select_dtypes(exclude=['number']).columns

# Fehlende Werte in den Trainingsdaten ersetzen
X_train[categoric_columns] = imputer2.fit_transform(X_train[categoric_columns])
print(categoric_columns)
# Fehlende Werte in den Testdaten ersetzen
X_test[categoric_columns] = imputer2.transform(X_test[categoric_columns])


Index(['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
       'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
       'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
       'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
       'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
       'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
       'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
       'PavedDrive', 'SaleType', 'SaleCondition'],
      dtype='object')


h) Wenden Sie One-Hot-Encoding auf die kategorischen Merkmale an.

Tip: OneHotEncoder(handle_unknown='ignore', sparse=False) sparse=False gibt eine voll besetzte Matrix zurück.

In [93]:
# your code here

categoric_columns = X.select_dtypes(exclude=['number']).columns
encoder=OneHotEncoder(handle_unknown='ignore', sparse=False)
X_train_ohe=pd.DataFrame(encoder.fit_transform(X_train[categoric_columns]))
X_test_ohe=pd.DataFrame(encoder.transform(X_test[categoric_columns]))
X_train_ohe.columns=encoder.get_feature_names_out(categoric_columns)
X_test_ohe.columns=encoder.get_feature_names_out(categoric_columns)
X_train_ohe.index = X_train.index
X_test_ohe.index = X_test.index
#X_train = pd.concat([X_train, X_train_ohe], axis=1)
#X_test = pd.concat([X_test, X_test_ohe], axis=1)
X_test = pd.concat([X_test.drop(categoric_columns, axis=1), pd.DataFrame(X_test_ohe)], axis=1)
X_train = pd.concat([X_train.drop(categoric_columns, axis=1), pd.DataFrame(X_train_ohe)], axis=1)
X_train


,MSSubClass,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,...,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_Abnorml,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial
Id,,,,,,,,,,,,,,,,,,,,,
255,20.0,70.0,8400.0,5.0,6.0,1957.0,1957.0,0.0,922.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1067,60.0,59.0,7837.0,6.0,7.0,1993.0,1994.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
639,30.0,67.0,8777.0,5.0,7.0,1910.0,1950.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
800,50.0,60.0,7200.0,5.0,7.0,1937.0,1950.0,252.0,569.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
381,50.0,50.0,5000.0,5.0,6.0,1924.0,1950.0,0.0,218.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1096,20.0,78.0,9317.0,6.0,5.0,2006.0,2006.0,0.0,24.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1131,50.0,65.0,7804.0,4.0,3.0,1928.0,1950.0,0.0,622.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
1295,20.0,60.0,8172.0,5.0,7.0,1955.0,1990.0,0.0,167.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


i) Trainieren Sie einen Randomforest Regressor auf den Trainingsdaten um die Immobilienpreise zu schätzen. Gehen Sie dabei folgendermaßen vor:

* Verwenden Sie eine 5-Fold Kreuzvalidierung um die Anzahl der Bäume zu finden, die das beste Ergebnis liefert (GridSearchCV). Probieren Sie die folgenden Konfigurationen aus: z.B. n_estimators: [50, 100, 150, 200]
* Wählen Sie die Konfiguration mit dem besten Ergebnis aus. Führe Sie diese auf den Testdaten aus und berechnen Sie den Mean Squared Error (MAE) sowie den R2_Score.


Tip: n_jobs=<number of Threads> für parallele Ausführung. 

In [95]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV

estimators =  [50, 100, 150, 200]

# your code here
rfe=RandomForestRegressor(n_estimators= 150, n_jobs=-1, random_state=42)
cv=GridSearchCV(estimator=rfe, cv=5, param_grid={'n_estimators': [50, 100, 150, 200]})
cv.fit(X_train, y_train)
y_pred=cv.predict(X_test)
r2score=r2_score(y_test, y_pred)
mae= mean_absolute_error(y_test, y_pred)

print(r2score)
print(mae)
print("best estimator: ", cv.best_params_)
#MAE: +17000
#r2_score=0.89

0.8916276409605626
17560.107579908676
best estimator:  {'n_estimators': 150}
